In [1]:
#THIS SHIT WORKS PLEASE DO NOT MAKE ANY CHANGES WHATSOEVER. 

#MAKE A COPY FIRST (hence "clean" version here)

In [2]:
# Package imports
import numpy as np
import sklearn

In [2]:
def softmax(x):
    """
    Computes softmax of input x
    
    Arguments:
    x -- input dataset of shape (input size, number of examples)

    Returns:
    s_x -- softmax of the vector x
    """
    exp_max = np.exp(x - np.max(x,axis=0,keepdims=True))
    s_x = exp_max/np.sum(exp_max,axis=0,keepdims=True)
    return s_x

In [3]:
def layer_sizes(X, n_h, Y):
    """
    Find the sizes of each layer
    
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    n_x = X.shape[0] # size of input layer
    n_y = Y.shape[0] # size of output layer
    return (n_x, n_h, n_y)

In [4]:
def initialize_parameters(n_x, n_h, n_y, seed=100):
    """
    Initialize weights as random numbers and bias as 0
    note 0.01 as "small number factor"
    
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(seed) # we set up a seed so that it will always return the same random parameters for testing
    W1 = np.random.randn(n_h, n_x)
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)
    b2 = np.zeros((n_y, 1))
    
    #sanity check
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [5]:
def forward_propagation(X, parameters):
    """
    Implement forward propagation
    
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The ssoftmax output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = np.dot(W1, X)+b1
    #relu
#     A1 = np.maximum(Z1,0)
    #this implementation of relu is faster
    A1 = Z1*(Z1>0)
    Z2 = np.dot(W2,A1)+b2
    #softmax
    A2 = softmax(Z2)
    
#     assert(A2.shape == (Y.shape[0], X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [6]:
def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost 
    
    """
    
    m = Y.shape[1]

    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(A2),Y)
    cost = - np.sum(logprobs)   
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [7]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    Z1 = cache['Z1']
    Z2 = cache['Z2']
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = Y-A2
    dW2 = 1/m*np.dot(dZ2, A1.T)
    db2 = 1/m*np.sum(dZ2, axis = 1, keepdims = True)
#     dZ1 = np.multiply(np.dot(W2.T, dZ2),(A1*(1-A1)))
    dZ1 = np.multiply(np.dot(W2.T, dZ2),np.heaviside(Z1,0))
    dW1 = 1/m*np.dot(dZ1, X.T)
    db1 = 1/m*np.sum(dZ1, axis = 1, keepdims = True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [15]:
def update_parameters(parameters, grads, learning_rate = 0.1):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Update rule for each parameter
    W1 = W1 + learning_rate*dW1
    b1 = b1 + learning_rate*db1
    W2 = W2 + learning_rate*dW2
    b2 = b2 + learning_rate*db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [9]:
def nn_model(X, Y, n_h, num_iterations = 10000, seed=100, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    n_x = layer_sizes(X, n_h, Y)[0]
    n_y = layer_sizes(X, n_h, Y)[2]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y, seed)
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [10]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model ('non-infectious illness': 0, 'bacterial': 1, 'viral': 2)
    """
    
    # Computes probabilities using forward propagation, and classifies to catefory using argmax
    A2, cache = forward_propagation(X, parameters)
    predictions = np.argmax(A2, axis = 0)
    
    return predictions

In [ ]:
# CLEAN run
# Accuracy for 0 hidden units: 57.14285714285714 %
# Accuracy for 1 hidden units: 75.0 %
# Accuracy for 2 hidden units: 71.42857142857143 %
# Accuracy for 3 hidden units: 71.42857142857143 %
# Accuracy for 4 hidden units: 71.42857142857143 %
# Accuracy for 5 hidden units: 67.85714285714286 %
# Accuracy for 6 hidden units: 64.28571428571429 %
# Accuracy for 7 hidden units: 71.42857142857143 %
# Accuracy for 8 hidden units: 71.42857142857143 %
# Accuracy for 9 hidden units: 67.85714285714286 %